In [10]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import NslKdd

In [11]:
transforms=None

#dataset
trainset=NslKddset("../../data","KDDTrain+")
testset=NslKddset("../../data","KDDTest+")

#dataloaders
trainloader=torch.utils.data.DataLoader(trainset,batch_size=10,shuffle=True)
testloader=torch.utils.data.DataLoader(testset,batch_size=10)

#constant for classes
classes=('normaly','anamaly')

TypeError: 'module' object is not callable

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(41,41*41)
        self.fc2=nn.Linear(41*41,666)
        self.fc3=nn.Linear(666,13)
        self.fc4=nn.Linear(13,2)

    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=self.fc4(x)
        return x
        

In [ ]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.90)
scheduler = optim.lr_scheduler.StepLR(optimizer, 1000,gamma=0.9)

In [ ]:
running_loss = 0.0
for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # every 1000 mini-batches...
            scheduler.step()
            print("[%d,%6d] ava_loss: %.5f"%(epoch+1,i+1,running_loss/1000))
            running_loss = 0.0
print('Finished Training')

In [ ]:
PATH="../../pth/nsl_kdd_test.pth"
torch.save(net.state_dict(),PATH)

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        cvdata, labels = data
        outputs = net(cvdata)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in testloader:
        cvdata, labels = data
        outputs = net(cvdata)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))